In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, MaxPooling1D, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt

targets = ['Avg_Temperature', 'Radiation', 'Rain_Amount', 'Wind_Speed', 'Wind_Direction_cos','Wind_Direction_sin']
features = ['kingdom_group', 'DayOfWeek_sin', 'DayOfWeek_cos', 'DayOfYear_sin',
            'DayOfYear_cos', 'Month_sin', 'Month_cos', 'Quarter_sin', 'Quarter_cos']

def create_sequences(data, seq_length):
    xs, ys = [], []
    for kingdom in data['kingdom_group'].unique():
        kingdom_data = data[data['kingdom_group'] == kingdom]
        kingdom_data = kingdom_data.sort_values('Date')
        X = kingdom_data[features].values
        y = kingdom_data[targets].values
        for i in range(len(X) - seq_length):
            xs.append(X[i:(i + seq_length)])
            ys.append(y[i + seq_length])
    return np.array(xs), np.array(ys)

seq_length = 30

train_df['Date'] = pd.to_datetime(train_df['Date'])
test_df['Date'] = pd.to_datetime(test_df['Date'])

feature_scaler = StandardScaler()
train_df[features] = feature_scaler.fit_transform(train_df[features])
test_df[features] = feature_scaler.transform(test_df[features])

target_scaler = StandardScaler()
train_df[targets] = target_scaler.fit_transform(train_df[targets])

X_train, y_train = create_sequences(train_df, seq_length)

def build_cnn_model(input_shape, output_dim):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(output_dim)
    ])

    model.compile(
        optimizer='adam',
        loss='mse'
    )
    return model

input_shape = (seq_length, len(features))
model = build_cnn_model(input_shape, len(targets))

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

def predict_for_test(model, test_df, train_df, seq_length):
    predictions = pd.DataFrame()

    for kingdom in test_df['kingdom_group'].unique():
        kingdom_train = train_df[train_df['kingdom_group'] == kingdom].sort_values('Date').tail(seq_length)
        kingdom_test = test_df[test_df['kingdom_group'] == kingdom].sort_values('Date')
        sequence = kingdom_train[features].values.astype(np.float32)

        kingdom_preds = []
        for _, row in kingdom_test.iterrows():
            X = np.array([sequence[-seq_length:]]).astype(np.float32)

            if X.shape[1:] != (seq_length, len(features)):
                if X.shape[1] < seq_length:
                    padding = np.zeros((1, seq_length - X.shape[1], X.shape[2]), dtype=np.float32)
                    X = np.concatenate([padding, X], axis=1)

            pred = model.predict(X, verbose=0)[0]
            kingdom_preds.append(pred)
            new_features = row[features].values.reshape(1, -1).astype(np.float32)
            sequence = np.vstack([sequence, new_features])

        kingdom_df = kingdom_test.copy()

        if kingdom_preds:
            kingdom_pred_array = np.array(kingdom_preds)

            for i, target in enumerate(targets):
                kingdom_df[target] = kingdom_pred_array[:, i]

            predictions = pd.concat([predictions, kingdom_df])

    return predictions

def fix_datatypes(df, columns):
    for col in columns:
        if df[col].dtype == 'object':
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                print(f"Could not convert {col} to float.")
    return df

train_df = fix_datatypes(train_df, features + targets)
test_df = fix_datatypes(test_df, features)

predictions_df = predict_for_test(model, test_df, train_df, seq_length)

predictions_array = predictions_df[targets].values
original_scale_predictions = target_scaler.inverse_transform(predictions_array)

for i, target in enumerate(targets):
    predictions_df[target] = original_scale_predictions[:, i]

val_size = int(0.1 * len(train_df))
val_df = train_df.tail(val_size)
train_df_reduced = train_df.iloc[:-val_size]

X_train_final, y_train_final = create_sequences(train_df_reduced, seq_length)
val_predictions = predict_for_test(model, val_df, train_df_reduced, seq_length)

metrics = {}
for i, target in enumerate(targets):
    actual = target_scaler.inverse_transform(val_df[targets].values)[:, i]
    predicted = target_scaler.inverse_transform(val_predictions[targets].values)[:, i]

    mae = mean_absolute_error(actual, predicted)
    mse = mean_squared_error(actual, predicted)
    rmse = sqrt(mse)
    r2 = r2_score(actual, predicted)
    mape = np.mean(np.abs((actual - predicted) / np.maximum(np.abs(actual), 1e-10))) * 100

    metrics[target] = {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R²': r2,
        'MAPE (%)': mape
    }

metrics_df = pd.DataFrame(metrics)
avg_metrics = metrics_df.mean(axis=1)

for target in targets:
    test_df[target] = predictions_df[target]